## DB와 연결하려 했으나 실패

In [9]:
import psycopg2

db_connect = psycopg2.connect(
        user="tasha",
        password="tasha",
        host="192.168.253.128",
        port=5432,
        database="tasha-progress-server",
        connect_timeout=3
)

OperationalError: connection to server at "192.168.253.128", port 5432 failed: timeout expired


## import & api test

In [1]:
import requests
import json
import pandas as pd
from tqdm import tqdm
import itertools
import pickle
import numpy as np

In [2]:
with open('./username_sample.pkl', 'rb') as f:
	username_list = pickle.load(f)

In [29]:
url = 'http://ws.audioscrobbler.com/2.0'

params = {
    "method": "user.getrecenttracks",
    "extended": 1,
    "user": "JM",
    "limit": 1,
    "api_key": "1a62f2d4937452d62d7426029e4c9997",
    "format": "json",
}

text = requests.get(url, params).text
texts = pd.DataFrame(json.loads(text)['recenttracks']['track'])

display(texts)
print(text)

,artist,date,mbid,name,image,url,streamable,album,loved
0,{'url': 'https://www.last.fm/music/my+bloody+v...,"{'uts': '1108290000', '#text': '13 Feb 2005, 1...",04e30386-e709-4a86-adc7-ca2ff8ed729a,Sometimes,"[{'size': 'small', '#text': ''}, {'size': 'med...",https://www.last.fm/music/my+bloody+valentine/...,0,"{'mbid': '', '#text': ''}",1


{"recenttracks":{"track":[{"artist":{"url":"https:\/\/www.last.fm\/music\/my+bloody+valentine","name":"my bloody valentine","image":[{"size":"small","#text":"https:\/\/lastfm.freetls.fastly.net\/i\/u\/34s\/2a96cbd8b46e442fc41c2b86b821562f.png"},{"size":"medium","#text":"https:\/\/lastfm.freetls.fastly.net\/i\/u\/64s\/2a96cbd8b46e442fc41c2b86b821562f.png"},{"size":"large","#text":"https:\/\/lastfm.freetls.fastly.net\/i\/u\/174s\/2a96cbd8b46e442fc41c2b86b821562f.png"},{"size":"extralarge","#text":"https:\/\/lastfm.freetls.fastly.net\/i\/u\/300x300\/2a96cbd8b46e442fc41c2b86b821562f.png"}],"mbid":""},"date":{"uts":"1108290000","#text":"13 Feb 2005, 10:20"},"mbid":"04e30386-e709-4a86-adc7-ca2ff8ed729a","name":"Sometimes","image":[{"size":"small","#text":""},{"size":"medium","#text":""},{"size":"large","#text":""},{"size":"extralarge","#text":""}],"url":"https:\/\/www.last.fm\/music\/my+bloody+valentine\/_\/Sometimes","streamable":"0","album":{"mbid":"","#text":""},"loved":"1"}],"@attr":{"us

In [64]:
import requests
import json
import pandas as pd
from tqdm import tqdm
import itertools

url = 'http://ws.audioscrobbler.com/2.0'

params = {
    "method": "user.getrecenttracks",
    "extended": 1,
    "user": "",
    "limit": 1,
    "api_key": "1a62f2d4937452d62d7426029e4c9997",
    "format": "json",
}

dataframe_list = []
for i, user in enumerate(tqdm(username_list)):
    params["user"] = user
    params["limit"] = 1
    text = requests.get(url, params).text
    text = json.loads(text)
    text = text['recenttracks']
    track = text['track']
    attr = text['@attr']
    params["limit"] = 200
    for page in range(int(attr['totalPages'])):
        params["page"] = page + 1
        text = requests.get(url, params).text
        texts = pd.DataFrame(json.loads(text)['recenttracks']['track'])
        texts['username'] = user
        texts['user_total'] = attr['total']

        texts['album_mbid'] = texts['album'].apply(lambda x: x['mbid'])
        texts['album_name'] = texts['album'].apply(lambda x: x['#text'])

        # texts['date_uts'] = texts['date'].apply(lambda x: np.NAN if x==np.NAN else x['uts'])
        texts['date_uts'] = texts['date'].apply(lambda x: x['uts'])
        texts['date_string'] = texts['date'].apply(lambda x: x['#text'])

        texts['artist_url'] = texts['artist'].apply(lambda x: x['url'])
        texts['artist_name'] = texts['artist'].apply(lambda x: x['name'])
        texts['artist_mbid'] = texts['artist'].apply(lambda x: x['mbid'])

        texts.drop(columns=['album', 'date', 'artist', 'image'], inplace=True)
        dataframe_list.append(texts)
        break
    if i > 10:
        break
data_csv = pd.concat(dataframe_list, ignore_index=True)
data_csv.rename({'mbid':'track_mbid', 'name':'track_name', 'url':'track_url'})
data_csv.replace('', np.NaN, inplace=True)
display(data_csv)

 11%|█         | 11/100 [00:12<01:42,  1.15s/it]


,mbid,name,url,streamable,loved,username,user_total,album_mbid,album_name,date_uts,date_string,artist_url,artist_name,artist_mbid
0,NaN,"Requiem in D Minor, K. 626: 4. Offertorium: Do...",https://www.last.fm/music/Wolfgang+Amadeus+Moz...,0,0,cynocauma,29607,NaN,Mozart: Requiem,1674264534,"21 Jan 2023, 01:28",https://www.last.fm/music/Wolfgang+Amadeus+Mozart,Wolfgang Amadeus Mozart,NaN
1,NaN,"Requiem in D Minor, K. 626: 3. Sequentia: Lacr...",https://www.last.fm/music/Wolfgang+Amadeus+Moz...,0,0,cynocauma,29607,NaN,Mozart: Requiem,1674264364,"21 Jan 2023, 01:26",https://www.last.fm/music/Wolfgang+Amadeus+Mozart,Wolfgang Amadeus Mozart,NaN
2,NaN,"Requiem in D Minor, K. 626: 3. Sequentia: Conf...",https://www.last.fm/music/Wolfgang+Amadeus+Moz...,0,0,cynocauma,29607,NaN,Mozart: Requiem,1674264207,"21 Jan 2023, 01:23",https://www.last.fm/music/Wolfgang+Amadeus+Mozart,Wolfgang Amadeus Mozart,NaN
3,NaN,"Requiem in D Minor, K. 626: 3. Sequentia: Reco...",https://www.last.fm/music/Wolfgang+Amadeus+Moz...,0,0,cynocauma,29607,NaN,Mozart: Requiem,1674263903,"21 Jan 2023, 01:18",https://www.last.fm/music/Wolfgang+Amadeus+Mozart,Wolfgang Amadeus Mozart,NaN
4,NaN,"Requiem in D Minor, K. 626: 3. Sequentia: Rex ...",https://www.last.fm/music/Wolfgang+Amadeus+Moz...,0,0,cynocauma,29607,NaN,Mozart: Requiem,1674263798,"21 Jan 2023, 01:16",https://www.last.fm/music/Wolfgang+Amadeus+Mozart,Wolfgang Amadeus Mozart,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,NaN,Sur le drapeau,https://www.last.fm/music/Supr%C3%AAme+NTM/_/S...,0,0,cynocauma,29607,NaN,Sur le drapeau (Extrait du projet 93 Empire),1674156906,"19 Jan 2023, 19:35",https://www.last.fm/music/Supr%C3%AAme+NTM,Suprême NTM,NaN
2396,NaN,Sur le drapeau,https://www.last.fm/music/Supr%C3%AAme+NTM/_/S...,0,0,cynocauma,29607,NaN,Sur le drapeau (Extrait du projet 93 Empire),1674156707,"19 Jan 2023, 19:31",https://www.last.fm/music/Supr%C3%AAme+NTM,Suprême NTM,NaN
2397,NaN,Sur le drapeau,https://www.last.fm/music/Supr%C3%AAme+NTM/_/S...,0,0,cynocauma,29607,NaN,Sur le drapeau (Extrait du projet 93 Empire),1674156507,"19 Jan 2023, 19:28",https://www.last.fm/music/Supr%C3%AAme+NTM,Suprême NTM,NaN
2398,NaN,Sur le drapeau,https://www.last.fm/music/Supr%C3%AAme+NTM/_/S...,0,0,cynocauma,29607,NaN,Sur le drapeau (Extrait du projet 93 Empire),1674156308,"19 Jan 2023, 19:25",https://www.last.fm/music/Supr%C3%AAme+NTM,Suprême NTM,NaN


## mbid 에 null 이 너무 많음 & user 끼리 중복됨

In [66]:
data_csv.notna().sum()
# mbid 에는 null 이 많아서 name 으로 대체

mbid           1307
name           2400
url            2400
streamable     2400
loved          2400
username       2400
user_total     2400
album_mbid     1396
album_name     2365
date_uts       2400
date_string    2400
artist_url     2400
artist_name    2400
artist_mbid       0
dtype: int64

In [23]:
from collections import Counter

result = Counter(username_list)
print(result)

for key in result:
    print(key, result[key])

result = Counter(username_list).values()
print(result)

Counter({'erickdsr12': 14, 'cynocauma': 13, 'NovicePlayer': 12, 'leyrey': 10, 'Nouwh': 10, 'Spectrum321146': 10, 'ZombieTeam67': 9, 'BilingualZebra7': 9, 'AetherJoako': 8, 'rataclans': 5})
cynocauma 13
AetherJoako 8
leyrey 10
NovicePlayer 12
Nouwh 10
erickdsr12 14
ZombieTeam67 9
BilingualZebra7 9
rataclans 5
Spectrum321146 10
dict_values([13, 8, 10, 12, 10, 14, 9, 9, 5, 10])


## iteraction 수집 & 각종 dictionary 들 선언

In [3]:
import requests
import json
import pandas as pd
from tqdm import tqdm
import itertools

url = 'http://ws.audioscrobbler.com/2.0'

params_inter = {
    "method": "user.getrecenttracks",
    "extended": 1,
    "user": "",
    "limit": 1,
    "api_key": "1a62f2d4937452d62d7426029e4c9997",
    "format": "json",
}

user2id = {}
track2id = {}
album2id = {}
artist2id = {}

track2artist = {}
artist2album = {}

album2track = {}

def push_track2id(str):
    if str not in track2id:
        track2id[str] = len(track2id)

def push_user2id(str):
    if str not in user2id:
        user2id[str] = len(user2id)

def push_album2id(str):
    if str not in album2id:
        album2id[str] = len(album2id)

def push_artist2id(str):
    if str not in artist2id:
        artist2id[str] = len(artist2id)

def push_track2artist(track, artist):
    if track not in track2artist:
        track2artist[track] = artist

def push_artist2album(artist, album):
    if isinstance(album, float) or album == '':
        return
    if artist not in artist2album:
        artist2album[artist] = [album]
    elif album not in artist2album[artist]:
        artist2album[artist].append(album)

def push_album2track(album, track):
    if album not in album2track:
        album2track[album] = [track]
    elif album not in album2track[album]:
        album2track[album].append(album)

for user in username_list:
    push_user2id(user)

# id2item, id2user dict 는 만들기가 까다로움. 업데이트가 까다롭다.

dataframe_list = []
for user, i in tqdm(user2id.items()):
    params_inter["user"] = user
    params_inter["limit"] = 1
    text = requests.get(url, params_inter).text
    text = json.loads(text)
    text = text['recenttracks']
    track = text['track']
    attr = text['@attr']
    params_inter["limit"] = 200
    for page in range(int(attr['totalPages'])):
        params_inter["page"] = page + 1
        text = requests.get(url, params_inter).text
        texts = json.loads(text)['recenttracks']['track']
        texts = pd.DataFrame(texts)
        texts['username'] = user
        texts['user_total'] = attr['total']

        texts['album_name'] = texts['album'].apply(lambda x: x['#text'])

        texts['date_uts'] = texts['date'].apply(lambda x: float('nan') if isinstance(x, float) else x['uts'])
        texts['date_string'] = texts['date'].apply(lambda x: float('nan') if isinstance(x, float) else x['#text'])

        texts['artist_name'] = texts['artist'].apply(lambda x: x['name'])

        texts['name'].apply(lambda x: push_track2id(x))
        texts['album_name'].apply(lambda x: push_album2id(x))
        texts['artist_name'].apply(lambda x: push_artist2id(x))
        texts.apply(lambda x: push_track2artist(x['name'], x['artist_name']), axis=1)
        texts.apply(lambda x: push_artist2album(x['artist_name'], x['album_name']), axis=1)

        texts.drop(columns=['album', 'date', 'artist', 'image', 'mbid', 'url', 'user_total', 'streamable'], inplace=True)
        if '@attr' in list(texts.keys()):
            texts.drop(columns=['@attr'], inplace=True)
        
        dataframe_list.append(texts)
        break
data_csv = pd.concat(dataframe_list, ignore_index=True)
data_csv.rename(columns={'name':'track_name'}, inplace=True)
data_csv.replace('', np.NaN, inplace=True)
data_csv = data_csv.astype({
    'track_name':'string',
    'loved':'int8',
    'username':'string',
    'album_name':'string',
    'date_uts':'string',
    'date_string':'string',
    'artist_name':'string',
})
display(data_csv)
data_csv.to_csv("interaction_2000.csv", index=False)

100%|██████████| 10/10 [00:11<00:00,  1.18s/it]


,track_name,loved,username,album_name,date_uts,date_string,artist_name
0,Ikuinen vappu,0,cynocauma,Ikuinen vappu,1674647452,"25 Jan 2023, 11:50",JVG
1,Ikuinen vappu,0,cynocauma,Ikuinen vappu,1674647073,"25 Jan 2023, 11:44",JVG
2,Ikuinen vappu,0,cynocauma,Ikuinen vappu,1674646883,"25 Jan 2023, 11:41",JVG
3,Ikuinen vappu,0,cynocauma,Ikuinen vappu,1674646694,"25 Jan 2023, 11:38",JVG
4,Ikuinen vappu,0,cynocauma,Ikuinen vappu,1674646503,"25 Jan 2023, 11:35",JVG
...,...,...,...,...,...,...,...
1996,Stopgap,0,Spectrum321146,Groove,1674490044,"23 Jan 2023, 16:07",StreamBeats by Harris Heller
1997,Disco Ball,0,Spectrum321146,Groove,1674489923,"23 Jan 2023, 16:05",StreamBeats by Harris Heller
1998,Groove All Night,0,Spectrum321146,Groove,1674489783,"23 Jan 2023, 16:03",StreamBeats by Harris Heller
1999,Rehearsal,0,Spectrum321146,Groove,1674489665,"23 Jan 2023, 16:01",StreamBeats by Harris Heller


In [5]:
data_csv.dtypes

track_name     string
loved            int8
username       string
album_name     string
date_uts       string
date_string    string
artist_name    string
dtype: object

In [4]:
data_csv[data_csv.album_name.isnull()].head()

,track_name,loved,username,album_name,date_uts,date_string,artist_name
439,Летучая мышь,0,leyrey,NaN,1674500543,"23 Jan 2023, 19:02",Филипп Киркоров
523,Opening 2 Full,0,leyrey,NaN,1674473847,"23 Jan 2023, 11:37",Death Note
561,I Need Air,0,leyrey,NaN,1674429812,"22 Jan 2023, 23:23",Nothing But Thieves
569,"Man [Blue Madonna, 2018]",0,leyrey,NaN,1674428025,"22 Jan 2023, 22:53",BØRNS
576,Звезда по имени солнце,0,leyrey,NaN,1674426397,"22 Jan 2023, 22:26","В. Цой, Кино"


In [103]:
print(len(artist2album)) # album 이 없는 아티스트도 존재하는 듯
print(len(artist2id))

816
838


In [91]:
# temp_list = [user2id, track2id, album2id, artist2id, track2artist]

# with open('dictionary_list_0122.pkl', 'wb') as f:
# 	pickle.dump(temp_list, f, protocol=pickle.HIGHEST_PROTOCOL)

## track_info

In [11]:
params_track = {
    "method": "track.getInfo",
    "track": "",
    "artist": "",
    "api_key": "1a62f2d4937452d62d7426029e4c9997",
    "autocorrect": 0,
    "format": "json",
}

tag2id = {}

def push_tag2id(tags):
    for tag in tags:
        if tag not in tag2id:
            tag2id[tag] = len(tag2id)

trackInfo_dataframe_list = []
for i, (track, artist) in enumerate(tqdm(track2artist.items())):
    params_track['track'] = track
    params_track['artist'] = artist
    track = requests.get(url, params_track).text
    track = json.loads(track)['track']

    track['artist_name'] = track['artist']['name']
    # if 'mbid' in list(track['artist'].keys()):     
    #     track['artist_mbid'] = track['artist'].apply(lambda x: x['mbid'])
    # else:
    #     track['artist_mbid'] = np.nan
    track['artist_url'] = track['artist']['url']

    # track['tag'] = track['toptags'].apply(lambda x: x['tag']) # 태그가 다 없음
    track['tag'] = list(pd.DataFrame(track['toptags']['tag']).apply(lambda x: (x['name']), axis=1))
    # print(track['tag'])
    push_tag2id(track['tag'])

    # track['toptags'].apply(lambda x: push_tag2id(x['tag']), axis=1)
    # track['tag'] = track['toptags'].apply(lambda x: x['tag'] if x != list([]) else np.nan)
    
    if 'album' in list(track.keys()):
        track['album_title'] = track['album']['title']
    #     track['album_url'] = track['album'].apply(lambda x: x['image'][-1]['#text'])
    #     track['album_artist'] = track['album'].apply(lambda x: x['artist'])
    #     track.drop(columns=['album'], inplace=True)
    else:
        track['album_title'] = np.nan
    #     track['album_url'] = np.nan
    #     track['album_artist'] = np.nan

    track = pd.DataFrame([track])
    if 'album' in list(track.keys()):
        track.drop(columns=['album'], inplace=True)
    
    if 'wiki' in list(track.keys()):
        track.drop(columns=['wiki'], inplace=True)

    if 'mbid' in list(track.keys()):
        track.drop(columns=['mbid'], inplace=True)
    track['streamable_text'] = track['streamable'].apply(lambda x: x['#text'])
    track['streamable_fulltrack'] = track['streamable'].apply(lambda x: x['fulltrack'])
    track.drop(columns=['artist', 'streamable', 'toptags'], inplace=True)
    trackInfo_dataframe_list.append(track)

    if i > 50:
        break

trackInfo_csv = pd.concat(trackInfo_dataframe_list, ignore_index=True)
trackInfo_csv.rename(columns={'name':'track_name', 'tag':'track_tag_list'}, inplace=True)
trackInfo_csv = trackInfo_csv.astype({
    'track_name':'string',
    'url':'string',
    'duration':'int32',
    'listeners':'int32',
    'playcount':'int32',
    'artist_name':'string',
    'artist_url':'string',
    'track_tag_list':'object',
    'album_title':'string',
    'artist_url':'string',
    'streamable_text':'int8',
    'streamable_fulltrack':'int8'
})
display(trackInfo_csv.head())
trackInfo_csv.to_csv("trackinfo_50.csv", index=False)

  4%|▍         | 51/1302 [00:13<05:29,  3.80it/s]


,track_name,url,duration,listeners,playcount,artist_name,artist_url,track_tag_list,album_title,streamable_text,streamable_fulltrack
0,Ikuinen vappu,https://www.last.fm/music/JVG/_/Ikuinen+vappu,0,11296,143005,JVG,https://www.last.fm/music/JVG,"[Suomirap, 2010s, 2019, Hip-Hop, pop]",Ikuinen vappu - Single,0,0
1,"Requiem in D Minor, K. 626: 8. Communio: Lux a...",https://www.last.fm/music/Wolfgang+Amadeus+Moz...,318000,5388,13057,Wolfgang Amadeus Mozart,https://www.last.fm/music/Wolfgang+Amadeus+Mozart,[],Deep Dive - Mozart,0,0
2,"Requiem in D Minor, K. 626: 7. Agnus Dei - Live",https://www.last.fm/music/Wolfgang+Amadeus+Moz...,209000,5928,14337,Wolfgang Amadeus Mozart,https://www.last.fm/music/Wolfgang+Amadeus+Mozart,[],Deep Dive - Mozart,0,0
3,"Requiem in D Minor, K. 626: 6. Benedictus - Live",https://www.last.fm/music/Wolfgang+Amadeus+Moz...,310000,5832,14298,Wolfgang Amadeus Mozart,https://www.last.fm/music/Wolfgang+Amadeus+Mozart,[],Deep Dive - Mozart,0,0
4,"Requiem in D Minor, K. 626: 5. Sanctus - Live",https://www.last.fm/music/Wolfgang+Amadeus+Moz...,127000,6476,15904,Wolfgang Amadeus Mozart,https://www.last.fm/music/Wolfgang+Amadeus+Mozart,[],Deep Dive - Mozart,0,0


In [12]:
trackInfo_csv.dtypes

track_name              string
url                     string
duration                 int32
listeners                int32
playcount                int32
artist_name             string
artist_url              string
track_tag_list          object
album_title             string
streamable_text           int8
streamable_fulltrack      int8
dtype: object

In [6]:
trackInfo_csv.notna().sum() # 수정 후: mbid, title, album, wiki, artist_mbid 는 삭제

track_name              52
url                     52
duration                52
listeners               52
playcount               52
artist_name             52
artist_url              52
track_tag_list          52
album_title             48
streamable_text         52
streamable_fulltrack    52
dtype: int64

In [7]:
(trackInfo_csv['streamable_fulltrack'] == trackInfo_csv['streamable_text']).sum()

52

## album info

In [14]:
params_album = {
    "method": "album.getInfo",
    "album": "",
    "artist" : "",
    "api_key": "1a62f2d4937452d62d7426029e4c9997",
    "autocorrect": 0,
    "format": "json",
}

albumInfo_dataframe_list = []
for i, (artist, albums) in enumerate(tqdm(artist2album.items())):
    params_album['artist'] = artist
    for album in albums:
        params_album['album'] = album
        album = requests.get(url, params_album).text
        try:
            album = json.loads(album)['album']
        except:
            print(album)
            print(artist, albums, album, params_album)
        
        album['tag'] = album['tags']
        # print(album['image'][-1])
        album['image'] = album['image'][-1]['#text']
        # print(album['wiki'])

        try:
            album['published'] = album['wiki']['published']
            # album['summary'] = album['wiki']['summary']
            album['content'] = album['wiki']['content']
            album = pd.DataFrame([album])
            album.drop(columns = ['wiki'], inplace=True)
        except:
            # album['summary'] = np.nan
            album['content'] = np.nan
            album = pd.DataFrame([album])

        # display(pd.DataFrame(album['tracks'].item()['track']))
        # album['track'] = list(album['tracks']['track'].apply(lambda x: (x['name'], x['artist']['name'])))
        # print(pd.DataFrame(album['tracks'].item()['track']))
        # print(album['tags'][0])
        try:
            if isinstance(album['tracks'].item()['track'], list):
                album['track'] = [list(pd.DataFrame(album['tracks'].item()['track']).apply(lambda x: (x['name'], x['artist']['name']), axis=1))]
            else:
                album['track'] = [list(pd.DataFrame([album['tracks'].item()['track']]).apply(lambda x: (x['name'], x['artist']['name']), axis=1))]
            album.drop(columns=['tracks'], inplace=True)
        except:
            album['track'] = np.nan # album 을 조회했는데 tracks 가 없는 경우..?
        try:
            album['tag'] = [list(pd.DataFrame(album['tags'].item()['tag']).apply(lambda x: (x['name']), axis=1))]
        except:
            album['tag'] = np.nan
            break
        albumInfo_dataframe_list.append(album)
        album.drop(columns=['tags', 'mbid'], inplace=True)
    if i > 50:
        break

albumInfo_csv = pd.concat(albumInfo_dataframe_list, ignore_index=True)
albumInfo_csv.rename(columns={'artist':'artist_name', 'name':'album_name'}, inplace=True)
albumInfo_csv.replace('', np.NaN, inplace=True)
albumInfo_csv = albumInfo_csv.astype({
    'artist_name':'string',
    'album_name':'string',
    'image':'string',
    'listeners':'int32',
    'playcount':'int32',
    'url':'string',
    'tag':'object',
    'content':'string',
    'track':'object',
    'published':'string',
})
display(albumInfo_csv)
albumInfo_csv.to_csv("albumInfo_50.csv", index=False)

  7%|▋         | 51/769 [00:40<09:25,  1.27it/s]  


,artist_name,album_name,image,listeners,playcount,url,tag,content,track,published
0,JVG,Ikuinen vappu,https://lastfm.freetls.fastly.net/i/u/300x300/...,7388,75613,https://www.last.fm/music/JVG/Ikuinen+vappu,"[suomirap, 2010s, 2019, pop, finnish]",<NA>,"[(Ikuinen vappu, JVG)]",<NA>
1,Wolfgang Amadeus Mozart,Mozart: Requiem,https://lastfm.freetls.fastly.net/i/u/300x300/...,133886,1669637,https://www.last.fm/music/Wolfgang+Amadeus+Moz...,"[classical, requiem, mozart, choral, conductor]",<NA>,"[(Requiem In D Minor, K.626 (Compl. By Franz X...",<NA>
2,NF,The Search,https://lastfm.freetls.fastly.net/i/u/300x300/...,322467,12478959,https://www.last.fm/music/NF/The+Search,"[rap, hip-hop, 2019, 2010s, emotional rap]","""The Search"" is the fourth studio album by Ame...","[(The Search, NF), (Leave Me Alone, NF), (Chan...","03 Mar 2022, 14:41"
3,Bry,BRY,https://lastfm.freetls.fastly.net/i/u/300x300/...,5651,107453,https://www.last.fm/music/Bry/BRY,"[rock, indie, indie rock, irish, ireland]",<NA>,"[(Disarm, Bry), (Hope You're Listening, Bry), ...",<NA>
4,Gringe,Enfant lune,https://lastfm.freetls.fastly.net/i/u/300x300/...,13994,371649,https://www.last.fm/music/Gringe/Enfant+lune,"[rap, france, french rap]",<NA>,"[(Mémo, Gringe), (Paradis noir (feat. DJ Pone)...",<NA>
...,...,...,...,...,...,...,...,...,...,...
61,Shawn Lee's Ping Pong Orchestra,Voices And Choices,https://lastfm.freetls.fastly.net/i/u/300x300/...,62264,530240,https://www.last.fm/music/Shawn+Lee%27s+Ping+P...,"[trip-hop, jazz, chillout, incomplete, snowboard]",<NA>,"[(Intro, Shawn Lee's Ping Pong Orchestra), (So...",<NA>
62,Megadeth,Killing Is My Business...And Business Is Good!,https://lastfm.freetls.fastly.net/i/u/300x300/...,99680,1421005,https://www.last.fm/music/Megadeth/Killing+Is+...,"[thrash metal, speed metal, 1985, 80s, metal]",Killing Is My Business…and Business Is Good! i...,NaN,"07 Jul 2022, 08:21"
63,Radiohead,OK Computer,https://lastfm.freetls.fastly.net/i/u/300x300/...,2867003,103323990,https://www.last.fm/music/Radiohead/OK+Computer,"[alternative rock, alternative, 1997, favourit...","""OK Computer"" is the third album by the Englis...","[(Airbag, Radiohead), (Paranoid Android, Radio...","11 Nov 2022, 15:31"
64,Avenged Sevenfold,City of Evil,https://lastfm.freetls.fastly.net/i/u/300x300/...,673153,20860919,https://www.last.fm/music/Avenged+Sevenfold/Ci...,"[metal, metalcore, hard rock, rock, heavy metal]",City of Evil is the third studio album by Aven...,"[(Beast and the Harlot, Avenged Sevenfold), (B...","07 Jul 2020, 13:42"


In [15]:
albumInfo_csv.dtypes

artist_name    string
album_name     string
image          string
listeners       int32
playcount       int32
url            string
tag            object
content        string
track          object
published      string
dtype: object

In [46]:
albumInfo_csv.notnull().sum()

artist_name    66
album_name     66
image          66
listeners      66
playcount      66
url            66
tag            66
content        35
track          64
published      35
dtype: int64

## userinfo

In [68]:
params_user = {
    "method": "user.getInfo",
    "user": "",
    "api_key": "1a62f2d4937452d62d7426029e4c9997",
    "format": "json",
}

userInfo_dataframe_list = []
for user, i in tqdm(user2id.items()):
    params_user['user'] = user
    user = requests.get(url, params_user).text
    user = json.loads(user)['user']
    user['image'] = user['image'][-1]['#text']
    user['registered'] = user['registered']['unixtime']
    user = pd.DataFrame([user])
    
    userInfo_dataframe_list.append(user)

userInfo_csv = pd.concat(userInfo_dataframe_list, ignore_index=True)
userInfo_csv.replace('None', np.NaN, inplace=True)
userInfo_csv.replace('', np.NaN, inplace=True)
userInfo_csv = userInfo_csv.astype({
    'name':'string',
    'age':'int',
    'subscriber':'int',
    'realname':'string',
    'bootstrap':'int',
    'playcount':'int',
    'artist_count':'int',
    'playlists':'int',
    'track_count':'int',
    'album_count':'int',
    'image':'string',
    'realname':'string',
    'registered':'string', # ?
    'country':'string',
    'gender':'string',
    'url':'string',
    'type':'string',
})
display(userInfo_csv)
userInfo_csv.to_csv("userInfo_10.csv", index=False) 

100%|██████████| 10/10 [00:03<00:00,  2.92it/s]


,name,age,subscriber,realname,bootstrap,playcount,artist_count,playlists,track_count,album_count,image,registered,country,gender,url,type
0,cynocauma,0,0,<NA>,0,29971,592,0,3286,1328,<NA>,1645178226,<NA>,n,https://www.last.fm/user/cynocauma,user
1,AetherJoako,0,0,Joaquín,0,7101,323,0,851,584,https://lastfm.freetls.fastly.net/i/u/300x300/...,1541372926,Chile,n,https://www.last.fm/user/AetherJoako,user
2,leyrey,0,0,leyrey,0,118381,2809,0,11101,5411,https://lastfm.freetls.fastly.net/i/u/300x300/...,1540131248,<NA>,n,https://www.last.fm/user/leyrey,user
3,NovicePlayer,0,0,<NA>,0,22899,974,0,4296,676,<NA>,1653624752,<NA>,n,https://www.last.fm/user/NovicePlayer,user
4,Nouwh,0,0,<NA>,0,51006,3149,0,8415,5946,https://lastfm.freetls.fastly.net/i/u/300x300/...,1571770571,<NA>,n,https://www.last.fm/user/Nouwh,user
5,erickdsr12,0,0,erick,0,217183,4842,0,14431,8796,https://lastfm.freetls.fastly.net/i/u/300x300/...,1427065817,Brazil,n,https://www.last.fm/user/erickdsr12,user
6,ZombieTeam67,0,0,<NA>,0,83645,1789,0,3597,2806,https://lastfm.freetls.fastly.net/i/u/300x300/...,1637281617,<NA>,n,https://www.last.fm/user/ZombieTeam67,user
7,BilingualZebra7,0,0,<NA>,0,20593,1711,0,4110,2524,<NA>,1641992153,<NA>,n,https://www.last.fm/user/BilingualZebra7,user
8,rataclans,0,0,<NA>,0,17427,1092,0,2697,1955,<NA>,1645507377,<NA>,n,https://www.last.fm/user/rataclans,user
9,Spectrum321146,0,0,<NA>,0,59185,2309,0,6608,3885,https://lastfm.freetls.fastly.net/i/u/300x300/...,1597755411,<NA>,n,https://www.last.fm/user/Spectrum321146,user


In [65]:
userInfo_csv.dtypes

name            string
age              int64
subscriber       int64
realname        string
bootstrap        int64
playcount        int64
artist_count     int64
playlists        int64
track_count      int64
album_count      int64
image           string
registered      string
country         string
gender          string
url             string
type            string
dtype: object

## taginfo

In [51]:
len(tag2id)

16

In [19]:
params_tag = {
    "method": "tag.getInfo",
    "tag": "",
    "api_key": "1a62f2d4937452d62d7426029e4c9997",
    "format": "json",
}

print(tag2id)

tagInfo_dataframe_list = []
for tag, i in tqdm(tag2id.items()):
    params_tag['tag'] = tag
    tag = requests.get(url, params_tag).text
    tag = json.loads(tag)['tag']
    # print(tag.keys())
    tag['summary'] = tag['wiki']['summary']
    tag['content'] = tag['wiki']['content']
    tag = pd.DataFrame([tag])
    tag.drop(columns=['wiki'], inplace=True)
    
    tagInfo_dataframe_list.append(tag)

tagInfo_csv = pd.concat(tagInfo_dataframe_list, ignore_index=True)
tagInfo_csv.replace('', np.NaN, inplace=True)
tagInfo_csv = tagInfo_csv.astype({
    'name':'string',
    'total':'int',
    'reach':'int',
    'summary':'string',
    'content':'string',
})
display(tagInfo_csv)
tagInfo_csv.to_csv("tagInfo.csv", index=False)

{'Suomirap': 0, '2010s': 1, '2019': 2, 'Hip-Hop': 3, 'pop': 4, 'rap': 5, 'conscious hip-hop': 6, 'trap': 7, 'rock': 8, 'fip': 9, 'Variete francaise': 10, 'Rap FR': 11, 'french': 12, 'indie': 13, 'indie pop': 14, 'chill': 15, 'hard rock': 16, 'alternative rock': 17, 'australian': 18, 'wolfmother': 19}


100%|██████████| 20/20 [00:12<00:00,  1.63it/s]


,name,total,reach,summary,content
0,Suomirap,6371,349,"Rap from Finland, also check out Suomi Rock. <...","Rap from Finland, also check out Suomi Rock. <..."
1,2010s,53656,1290,"Music released from 2010 to the end of 2019, a...","Music released from 2010 to the end of 2019, a..."
2,2019,19725,644,"Music released in the year 2019, and non-2019 ...","Music released in the year 2019, and non-2019 ..."
3,Hip-Hop,891974,129236,"Hip-Hop, also spelled Hip Hop or HipHop is a r...","Hip-Hop, also spelled Hip Hop or HipHop is a r..."
4,pop,2023130,229785,Pop music is a broad-term for many different t...,Pop music is a broad-term for many different t...
5,rap,595741,107031,"Rap is a vocal style, usually coming together ...","Rap is a vocal style, usually coming together ..."
6,conscious hip-hop,2723,566,Positive hip hop (also conscious rap or politi...,Positive hip hop (also conscious rap or politi...
7,trap,50132,9260,Trap music is a sub-genre of hip-hop that orig...,Trap music is a sub-genre of hip-hop that orig...
8,rock,4019591,398958,Rock music is a genre of music started in Amer...,Rock music is a genre of music started in Amer...
9,fip,204387,444,FIP (originally France Inter Paris) is a Frenc...,FIP (originally France Inter Paris) is a Frenc...


In [18]:
display(tagInfo_csv.dtypes)
tagInfo_csv = tagInfo_csv.astype({
    'name':'string',
    'total':'int',
    'reach':'int',
    'summary':'string',
    'content':'string',
})
display(tagInfo_csv.head())
display(tagInfo_csv.dtypes)

name       string
total       int64
reach       int64
summary    string
content    string
dtype: object

,name,total,reach,summary,content
0,Suomirap,6371,349,"Rap from Finland, also check out Suomi Rock. <...","Rap from Finland, also check out Suomi Rock. <..."
1,2010s,53656,1290,"Music released from 2010 to the end of 2019, a...","Music released from 2010 to the end of 2019, a..."
2,2019,19725,644,"Music released in the year 2019, and non-2019 ...","Music released in the year 2019, and non-2019 ..."
3,Hip-Hop,891974,129236,"Hip-Hop, also spelled Hip Hop or HipHop is a r...","Hip-Hop, also spelled Hip Hop or HipHop is a r..."
4,pop,2023130,229785,Pop music is a broad-term for many different t...,Pop music is a broad-term for many different t...


name       string
total       int64
reach       int64
summary    string
content    string
dtype: object